In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
np.set_printoptions(precision=2)
from cycler import cycler
plt.rcParams.update(plt.rcParamsDefault)

import wave as w
import constants as c
import material as m

# Example usage of simulated materials
This notebook is ment as an reference on how to use `PlaneWave` and `Material` classes,
and for visual test of the implementation.

### Generate an example spectrum
Just an example how to create and plot wave.

In [ ]:
s = w.GaussianPlanarWave(mean=c.GREEN, std=(5*c.MICRO))
# s = w.DeltaPlanarWave(wavelength=c.GREEN)
z = np.linspace(-6*c.MICRO, 6*c.MICRO, 1000)
s.delay(0)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
s.plot(ax=ax1, spectrum_axis=ax1.twinx(), wavelength=True)
s.plot_amplitude(z, ax=ax2)
ax2.legend()
fig.tight_layout()
plt.savefig("spectrum.pdf")
plt.show()

### Compare different materials

#### Analitical vs constant vs matrix theory
Create the same material (empty space) using three different material models in oder to inspect that they give the same resut.

In [ ]:
air = m.EmptySpace(z=z, name="analytic")
common_n = 1
fixed_dielectric = m.ConstantMaterial(z=z, n0=common_n, name="phase shift")
dielectric = m.SimpleDielectric(z=z, n0=1, name="transfer matrices")

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

air.plot(ax1, imaginary_axis=ax1.twinx())
fixed_dielectric.plot(ax2, imaginary_axis=ax2.twinx())
dielectric.plot(ax3, imaginary_axis=ax3.twinx())
fig.tight_layout() 
plt.show()

Calculate interference pattern in those three materials: again we can see that the patterns match. 
The index of refraction after recording would be different in those materials because ConstantMaterial's index of refraction cannot be modified.

In [ ]:
fig, (ax1) = plt.subplots(1, 1, figsize=(8,4))

air.record(s,s)
air.plot_recent_energy(ax1, label="air")
dielectric.record(s,s)
dielectric.plot_recent_energy(ax1, label="matrix theory")
fixed_dielectric.record(s,s)
fixed_dielectric.plot_recent_energy(ax1, label="correlation")
ax1.legend()
plt.show()

### An example recording

Left: interference pattern 
created in the material. Middle: index of 
refraction after recording. Right: reflected pulse, when the 
incoming pulse was the same as the recorded.

In [ ]:
z = np.linspace(-6*c.MICRO, 6*c.MICRO, 1000)
dielectric = m.SimpleDielectric(z=z, n0=1.45, name="transfer matrices")
s0 = w.GaussianPlanarWave(mean=c.GREEN, std=(5*c.MICRO))
dielectric.record(s0, s0)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
dielectric.plot_recent_energy(ax1)
ax1.set_title("interference")
dielectric.plot(imaginary_axis=ax2.twinx(), ax=ax2)
dielectric.reflect(s0).plot(ax3, spectrum_axis=ax3.twinx(), wavelength=True)
plt.tight_layout()
plt.savefig("recording.pdf")
plt.show()

### Layered material

Left: interference 
pattern 
created in the material. Middle: index of refraction after recording. Right: 
reflected pulse. All inner implementations are the same as for dielectric.
Additionally, on the first two plots shading is 
added to indicate layers of different index of 
refraction.

In [ ]:
z_layered = np.linspace(-2*c.MICRO, 2*c.MICRO, 1000)
layered = m.LayeredMaterial(z=z_layered, n_layers=20, layer_width=80*c.NANO, n0=1.5, n1=2.0)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
layered.shade_plot(ax1, alpha=0.2)
layered.plot(ax1, imaginary_axis=ax1.twinx())
layered.shade_plot(ax2, alpha=0.2)
layered.reflect(s).plot(ax3, label="reflected", wavelength=True, spectrum_axis=ax3.twinx())
layered.record(s, s)
layered.plot_recent_energy(ax2)
fig.tight_layout() 
plt.savefig("layered.pdf")
plt.show()

### Composite Material

Left: 
interference pattern 
created in the material. Middle: index of refraction after recording. Right: 
reflected pulse. All inner implementations are the same as for dielectric. Again, the shading is 
added to indicate different components. The strange reflected wave shape comes mostly from interfaces with air which haven't been modeled before.

In [ ]:
z_layered = np.linspace(-2*c.MICRO, 2*c.MICRO, 1000)
z_air = np.linspace(-0.4*c.MICRO, 0, 100)
z_clean = np.linspace(-1.6*c.MICRO, 0, 400)
air = m.EmptySpace(z=z_air)
clean = m.SimpleDielectric(z=z_clean, n0=1.5)
layered = m.LayeredMaterial(z=z_layered, n_layers=20, layer_width=80*c.NANO, n0=1.5, n1=2.0)
composite = m.CompositeMaterial([air, layered, clean, air])

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
composite.shade_plot(ax1, alpha=0.2)
composite.plot(ax1, imaginary_axis=ax1.twinx())
composite.shade_plot(ax2, alpha=0.2)
composite.reflect(s).plot(ax3, label="reflected", wavelength=True, spectrum_axis=ax3.twinx())
composite.record(s, s)
composite.plot_recent_energy(ax2)
fig.tight_layout() 
plt.savefig("composite.pdf")
plt.show()

### Random phase example

Random phase. One pulse is kept the same, and the phase of the second 
one is choose uniformly at random form interval $(0, \pi)$. On the top left 
plot we can see that the interference patter indeed get blurred, and on the top 
right we can see that there is indeed a Gaussian shape emerging at the basis of 
the interference pattern. Fortunately, the patter still has a lot of variance at 
the right frequency, and thus with each pulse more gets reflected, as depicted 
at the bottom plots. We can see that there gain is not the same with every 
pulse.

In [ ]:
steps = 20
colors = cycler('color', plt.get_cmap('viridis')(np.linspace(0, 1, steps)))
colors = colors.by_key()['color']

In [ ]:
dielectric = m.SimpleDielectric(z=z, n0=1)
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))

for pulse, color in enumerate(colors):
    phase = np.random.uniform(0, np.pi)
    s2 = s0 * np.exp(1j*phase)
    dielectric.plot(ax=ax2, alpha=0.2, color=color)
    dielectric.record(s0, s2)
    dielectric.plot_recent_energy(ax1, label="pulse {}".format(pulse + 1), alpha=0.2, c=color)
    ax1.set_title("interference")
    dielectric.reflect(s0).plot(ax3, spectrum_axis=ax4, alpha=0.5, color=color)
    ax3.set_title("reflected")
    ax4.set_title("reflected")
        
plt.tight_layout()
plt.savefig("random_phase.pdf")
plt.show()

### Sweeping phase example

Random phase. One pulse is kept the same, and the phase of the second 
one is changing by $\pi/20$ with each pulse. On the top left 
plot we can see that the interference patter shifts to the right, and on the top 
right we can see the refractive index pattern is much sharper than in the previous example. The total reflection is however very similar. It seems 
to reach saturation faster, and be slightly sharper in wavelength.

In [ ]:
dielectric = m.SimpleDielectric(z=z, n0=1)
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))

for pulse, color in enumerate(colors):
    phase = np.pi*pulse/steps
    s2 = s0 * np.exp(1j*phase)
    dielectric.plot(ax=ax2, alpha=0.2, color=color)
    dielectric.record(s0, s2)
    dielectric.plot_recent_energy(ax1, label="pulse {}".format(pulse + 1), alpha=0.2, c=color)
    ax1.set_title("interference")
    dielectric.reflect(s0).plot(ax3, spectrum_axis=ax4, alpha=0.5, color=color)
    ax3.set_title("reflected")
    ax4.set_title("reflected")

plt.tight_layout()
plt.savefig("sweeping_phase.pdf")      
plt.show()

### Chirps example

#### Different chirps interfering with their copies. 
Left: chirping Gaussian spectrum leads to spreading of the pulse amplitude in 
time (and thus in space). Middle and right: the recorded patterns and reflected 
spectra look the same across the experiments.

In [ ]:
z = np.linspace(-6*c.MICRO, 6*c.MICRO, 2000)
probing_wave = w.GaussianPlanarWave(mean=c.GREEN, std=(5*c.MICRO))
for chirp in np.linspace(0, 0.002*c.NANO, 3):
    dielectric = m.SimpleDielectric(z=z, n0=1.45, name="transfer matrices")
    chirped_wave = w.ChirpedPlanarWave(mean=c.GREEN, std=(5*c.MICRO), skew=chirp)
    dielectric.record(chirped_wave, chirped_wave)
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    chirped_wave.plot_amplitude(z, ax=ax1)
    ax1.set_title("chirp {:2.1e}".format(chirp))
    dielectric.plot(imaginary_axis=ax2.twinx(), ax=ax2)
    dielectric.reflect(probing_wave).plot(ax3, spectrum_axis=ax3.twinx())
    plt.tight_layout()
    plt.savefig("chirp_only_{:2.0e}.pdf".format(chirp))
    plt.show()

#### Chirps interfering with a non chirped pulse.

Left: interference 
patterns look change if only one of the pulses is chirped. Right and middle:
the recorded pattern and the reflected spectrum differ from plot to plot. 
It is hard to tell how significant this change is.

In [ ]:
z = np.linspace(-6*c.MICRO, 6*c.MICRO, 2000)
probing_wave = w.GaussianPlanarWave(mean=c.GREEN, std=(5*c.MICRO))
for chirp in np.linspace(0, 0.002*c.NANO, 3):
    dielectric = m.SimpleDielectric(z=z, n0=1.45, name="transfer matrices")
    chirped_wave = w.ChirpedPlanarWave(mean=c.GREEN, std=(5*c.MICRO), skew=chirp)
    dielectric.record(chirped_wave, probing_wave)
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    dielectric.plot_recent_energy(ax1)
    ax1.set_title("interference (chirp {:2.1e})".format(chirp))
    dielectric.plot(imaginary_axis=ax2.twinx(), ax=ax2)
    dielectric.reflect(probing_wave).plot(ax3, spectrum_axis=ax3.twinx())
    plt.tight_layout()
    plt.savefig("chirp_mixed_{:2.0e}.pdf".format(chirp))
    plt.show()